In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("udf").getOrCreate()

In [3]:
transactions = [
    ('찹쌀탕수육+짜장2', '2021-11-07 13:20:00', 22000, 'KRW'),
    ('등심탕수육+크립새우+짜장면', '2021-10-24 11:19:00', 21500, 'KRW'), 
    ('월남 쌈 2인 세트', '2021-07-25 11:12:40', 42000, 'KRW'), 
    ('콩국수+열무비빔국수', '2021-07-10 08:20:00', 21250, 'KRW'), 
    ('장어소금+고추장구이', '2021-07-01 05:36:00', 68700, 'KRW'), 
    ('족발', '2020-08-19 19:04:00', 32000, 'KRW'),  
]

schema = ["name", "datetime", "price", "currency"]

In [4]:
df = spark.createDataFrame(data=transactions, schema=schema)

In [5]:
df.createOrReplaceTempView("transactions")

In [7]:
spark.sql("SELECT * FROM transactions").show()

+--------------------------+-------------------+-----+--------+
|                      name|           datetime|price|currency|
+--------------------------+-------------------+-----+--------+
|          찹쌀탕수육+짜장2|2021-11-07 13:20:00|22000|     KRW|
|등심탕수육+크립새우+짜장면|2021-10-24 11:19:00|21500|     KRW|
|          월남 쌈 2인 세트|2021-07-25 11:12:40|42000|     KRW|
|       콩국수+열무비빔국수|2021-07-10 08:20:00|21250|     KRW|
|       장어소금+고추장구이|2021-07-01 05:36:00|68700|     KRW|
|                      족발|2020-08-19 19:04:00|32000|     KRW|
+--------------------------+-------------------+-----+--------+



In [8]:
# UDF 

In [14]:
# from pyspark.sql.types import LongType
from pyspark.sql.functions import udf

@udf("long")

def squared(n):
    return n * n
#spark.udf.register("squared", squared, LongType())

In [15]:
spark.sql("SELECT price, squared(price) FROM transactions").printSchema()

root
 |-- price: long (nullable = true)
 |-- squared(price): long (nullable = true)



In [18]:
# 가격을 한국어로 읽어주는 함수 만들기 
def read_number(n):
    units = ["", "십", "백", "천", "만"]
    nums = '일이삼사오육칠팔구'
    # string값을 모아두는 배열
    result = []
    i = 0 
    while n > 0:
        # n을 넣었을때 10으로 나누면 결과값과 나머자가 뭐가 나오는지 확인할 수 있다.
        n, r = divmod(n, 10) 
        # 만약 나머지가 있을때 
        if r > 0:
            result.append(nums[r-1]+units[i])
        i += 1 
    return "".join(reversed(result))

print(read_number(21250))
print(read_number(68700))

이만일천이백오십
육만팔천칠백


In [19]:
spark.udf.register("read_number", read_number)

<function __main__.read_number(n)>

In [21]:
spark.sql("SELECT price, read_number(price) FROM transactions").show()

+-----+------------------+
|price|read_number(price)|
+-----+------------------+
|22000|          이만이천|
|21500|      이만일천오백|
|42000|          사만이천|
|21250|  이만일천이백오십|
|68700|      육만팔천칠백|
|32000|          삼만이천|
+-----+------------------+



In [22]:
# 요일을 읽어주는 함수 

In [34]:
def get_weekday(date):
    import calendar 
    return calendar.day_name[date.weekday()]

spark.udf.register("get_weekday", get_weekday)

<function __main__.get_weekday(date)>

In [36]:
query = """
SELECT 
    datetime,
    get_weekday(TO_DATE(datetime)) AS day_of_week
FROM 
    transactions
"""
spark.sql(query).show()

+-------------------+-----------+
|           datetime|day_of_week|
+-------------------+-----------+
|2021-11-07 13:20:00|     Sunday|
|2021-10-24 11:19:00|     Sunday|
|2021-07-25 11:12:40|     Sunday|
|2021-07-10 08:20:00|   Saturday|
|2021-07-01 05:36:00|   Thursday|
|2020-08-19 19:04:00|  Wednesday|
+-------------------+-----------+

